In [110]:
import glob

import tensorflow as tf
#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

import os
import time
from datetime import datetime
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt

params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (10, 7),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

workerEnv = str(os.getenv("USER"))
workOnPrometheus = workerEnv.find("plg")>-1
inputDataPrefix = "/home/user1/scratch/akalinow/"
if workOnPrometheus:
    inputDataPrefix = "/net/people/plgakalinow/plggcmsml/"
dataDir = inputDataPrefix+"/ProgrammingProjects/MachineLearning/TauTauMass/data/"   

massBins = np.concatenate((np.arange(0.0, 10, 1), np.arange(10, 100, 1), np.arange(100, 1000, 100), [99999.0]))

In [114]:
print(massBins.shape)
print(massBins[109])
np.searchsorted(massBins, (910), side='left')

(110,)
99999.0


109

## Data manipulation functions

In [76]:
columns = np.array(["genMass", "visMass", "caMass", "fastMTT",
                   "leg1_e", "leg1_px","leg1_py","leg1_pz", 
                   "leg2_e", "leg2_px","leg2_py","leg2_pz",
                   "leg_2_decayMode",
                    "met", "met_x", "met_y"])
                    
def getFeaturesMask():
    featuresMask = np.full_like(columns, True, dtype=np.bool)
    featuresMask *= columns!="genMass"    
    featuresMask *= columns!="visMass"
    featuresMask *= columns!="caMass"
    featuresMask *= columns!="fastMTT"
    return featuresMask                   

def mass2Label(tensor):
    tensor = tf.searchsorted(massBins, tensor, side='left')
    return tensor
    
def label2Mass(tensor):  
    return tf.where(massBins[tensor]<1000, massBins[tensor], [1000]) 

def modifyFeatures(dataRow, batchSize, isTrain=False):
    columnsMask = getFeaturesMask()
    features = tf.boolean_mask(dataRow, columnsMask, axis=1)
    
    columnIndex = np.where(columns == "genMass")[0][0]
    labels = dataRow[:,columnIndex]
    labels.set_shape([batchSize,])
    
    if isTrain:
        labels = mass2Label(labels)
        return (features, (labels))
    else:
        columnIndex = np.where(columns == "fastMTT")[0][0]  
        fastMTT = dataRow[:,columnIndex]
        fastMTT.set_shape([batchSize,])
        return (features, (labels), fastMTT)
    return dataRow
    
def loadDataset(filePaths, isTrain, nEpochs=1, batchSize=1):   
    parquetFile = filePaths[0]
    df = pd.read_parquet(parquetFile)
    dataset = tf.data.Dataset.from_tensor_slices(df.values)
    dataset = dataset.batch(batchSize, drop_remainder=True)
    dataset = dataset.map(lambda x: modifyFeatures(x, batchSize, isTrain))
    return dataset

def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(1E-10)
    tf.print("Execution time:", time.perf_counter() - start_time)

In [115]:
trainFileNames = glob.glob(dataDir + 'df.parquet_htt_features_train.gzip')
train_dataset = loadDataset(trainFileNames, isTrain=True, nEpochs=1, batchSize=1024)

dataset = train_dataset
#benchmark(dataset)
#benchmark(dataset.prefetch(tf.data.experimental.AUTOTUNE))

for element in train_dataset.take(10): 
  print(element[1][496]) 

tf.Tensor(101, shape=(), dtype=int32)
tf.Tensor(58, shape=(), dtype=int32)
tf.Tensor(83, shape=(), dtype=int32)
tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(101, shape=(), dtype=int32)
tf.Tensor(102, shape=(), dtype=int32)
tf.Tensor(101, shape=(), dtype=int32)
tf.Tensor(109, shape=(), dtype=int32)
tf.Tensor(56, shape=(), dtype=int32)
tf.Tensor(101, shape=(), dtype=int32)


## Define models

In [95]:
def getModel():
    
  nMassBins =  massBins.shape[0]
  print(nMassBins)
  nInputs = 12
  inputs = tf.keras.Input(shape=(nInputs,), name="features")
  activation = tf.keras.layers.Activation(tf.nn.leaky_relu)
  x = inputs
  for iLayer in range(0,10):
            x = tf.keras.layers.Dense(128, activation=activation, name="layer_{}".format(iLayer))(x)
            
  outputs = tf.keras.layers.Dense(nMassBins, activation=tf.nn.softmax,
                               bias_initializer='zeros',
                               name = "mass")(x)              
  model = tf.keras.Model(inputs=inputs, outputs=outputs, name="NN_SVFit")

  initial_learning_rate = 0.005
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=200,
    decay_rate=0.95,
    staircase=True)
  
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
  #tf.keras.utils.plot_model(model, 'fig_png/ML_model.png', show_shapes=True)
  return model

In [96]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

trainFileNames = glob.glob(dataDir + 'df.parquet_htt_features_train.gzip')
validationFileNames = glob.glob(dataDir + 'df.parquet_htt_features_train.gzip')

train_dataset = loadDataset(trainFileNames, isTrain=True, nEpochs=1, batchSize=1024)
validation_dataset = loadDataset(trainFileNames, isTrain=True, nEpochs=1, batchSize=1024)

model = getModel()
nEpochsSaved = 20
#checkpoint_path = "training/model_weights_{epoch:04d}.ckpt"
#model.load_weights(checkpoint_path.format(epoch=nEpochsSaved)) 

classes = np.arange(ptBins.shape[0])
weights = 1*(classes>40) + 1
class_weight = dict(zip(classes, weights))
print(class_weight)
nEpochs = 2
history = model.fit(train_dataset, epochs=nEpochs,
                    use_multiprocessing=True,
                    verbose=1,
                    shuffle=False,
                    validation_data=validation_dataset.take(10),
                    class_weight=class_weight
                   )

Current Time = 12:47:16
110
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 2, 52: 2, 53: 2, 54: 2, 55: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 2, 61: 2, 62: 2, 63: 2, 64: 2, 65: 2, 66: 2, 67: 2, 68: 2, 69: 2, 70: 2, 71: 2, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 82: 2, 83: 2, 84: 2, 85: 2, 86: 2, 87: 2, 88: 2, 89: 2, 90: 2, 91: 2, 92: 2, 93: 2, 94: 2, 95: 2, 96: 2, 97: 2, 98: 2, 99: 2, 100: 2, 101: 2, 102: 2, 103: 2, 104: 2, 105: 2, 106: 2, 107: 2, 108: 2}
Train for 321 steps, validate for 10 steps
Epoch 1/2
  7/321 [..............................] - ETA: 2:25 - loss: 9.1855 - accuracy: 0.3184 

InvalidArgumentError:  indices[496] = 109 is not in [0, 109)
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]] [Op:__inference_distributed_function_1042741]

Function call stack:
distributed_function
